In [15]:
import io
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import get_dataset_dir
from collections import defaultdict


min_films = 5
knn = 4

def get_info():
    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    info = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as file:
        for line in file:
            line = line.split('|')
            info[line[0]] = (line[1], line[2])
    return info

def rnk_surprise(usr):
    data = Dataset.load_builtin('ml-100k')
    trainset = data.build_full_trainset()

    sim_options = {'name': 'cosine', 'user_based': True, 'min_support': min_films}
    algo = KNNWithMeans(k = knn, min_k = knn, sim_options = sim_options)
    algo.fit(trainset)

    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == usr, testset)
    predictions = algo.test(testset)
    top_n = defaultdict(list)

    for uid, iid, _, est, _ in predictions:
        top_n[uid].append((iid, round(est, 3)))

    for uid, user_rnk in top_n.items():
        user_rnk.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_rnk[:min_films]

    info = get_info()
    
    query = []
    row_format = "{:^10} {:<70} {:^1}"
    
    print("Пользователь: " + str(usr))
    
    
    for film_rid, rnk in top_n[usr]:
        print(row_format.format(film_rid, str(info[film_rid]), rnk))
        
        #вырежем год из названия фильма
        film_year = re.findall(r"\d{4}", info[film_rid][0])
        
        #уберем год и артикли из названия фильма 
        if info[film_rid][0].find(" (") != -1:
            at = info[film_rid][0].find(" (")
            film = info[film_rid][0][:at]
            
        if info[film_rid][0].find(", The") != -1:
            at = info[film_rid][0].find(", The")
            film = info[film_rid][0][:at]
            
        if info[film_rid][0].find(", Das") != -1:
            at = info[film_rid][0].find(", Das")
            film = info[film_rid][0][:at]
            
        query.append([film, film_year[0]])
    return query

usr = input("Введите id пользователя: ")
query = rnk_surprise(usr)

Введите id пользователя: 2
Computing the cosine similarity matrix...
Done computing similarity matrix.
Пользователь: 2
   1512    ('World of Apu, The (Apur Sansar) (1959)', '05-Apr-1996')              5
   1367    ('Faust (1994)', '01-Jan-1994')                                        4.862
   1449    ('Pather Panchali (1955)', '22-Mar-1996')                              4.854
   1142    ('When We Were Kings (1996)', '14-Feb-1997')                           4.825
   1176    ('Welcome To Sarajevo (1997)', '01-Jan-1997')                          4.786
